In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import accuracy_score , precision_score

In [2]:
import os, sys, gc, warnings, random, datetime, math

In [3]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [4]:
SEED = 42
seed_everything(SEED)

In [5]:
train = pd.read_csv('PJT002_train.csv')
val = pd.read_csv('PJT002_validation.csv')
test = pd.read_csv('PJT002_test.csv')
sub = pd.read_csv('PJT002_submission.csv')
data = pd.concat([train, val, test])

In [6]:
def make_copy(data):
    data_ = data.copy()
    return data_

In [7]:
data['name_1'] = data['emd_nm'].str[5:8]

data['name_4'] = data['name_1'].str[2]

data['name_2'] = data['emd_nm'].str[::-1].str[:].str[2]+data['emd_nm'].str[::-1].str[:].str[1]+data['emd_nm'].str[::-1].str[:].str[0]

data['name_3'] =data['emd_nm'].str[::-1].str[:].str[0]

In [8]:
rm_cols = ['id']

In [36]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat)
    return 'f1_score', f1_score(y_true, y_hat), True

def lgbmgo(data_, train, test, rm_cols):

    train_ = data_[:len(train)+2500]
    val_ = data_[len(train)+2500:-len(test)]
    test_ = data_[-len(test):]
    dataset = [train_, val_, test_]

    for df in dataset:
        categorical_cols1 = df.select_dtypes(['object']).columns
        for col in categorical_cols1:
            df[col] = pd.Categorical(df[col]).codes
    train_.drop(rm_cols,1,inplace=True)
    val_.drop(rm_cols,1,inplace=True)
    test_.drop(rm_cols,1,inplace=True)
    X_train = train_.drop('fr_yn', 1)
    y_train = train_['fr_yn']
    X_val = val_.drop('fr_yn', 1)
    y_val = val_['fr_yn']
    X_test = test_.drop(columns = ['fr_yn'])
    
    print(X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape)
    #모델 생성 및 학습
    params = {
                    'boosting_type':'gbdt',
                    'n_jobs':-1,
                    'learning_rate':0.01,
                    'num_leaves': 250,
                    'max_depth':8,
                    'n_estimators':10000,
                    'seed' :SEED,
                }
    
    lgb_train = lgb.Dataset(X_train ,label= y_train)
    lgb_eval = lgb.Dataset(X_val, label= y_val)
    y_prob = np.zeros((X_test.shape[0]))

    model = lgb.train(params, lgb_train, feval=lgb_f1_score, valid_sets=[lgb_train, lgb_eval], early_stopping_rounds=300, verbose_eval=100)
    
    y_pred = model.predict(X_val)[:]
    val_pred = pd.DataFrame(y_pred)
    val_pred[0] = val_pred[0]>0.5
    val_pred[0] = val_pred[0].map({False:0,True:1})

    score = precision_score(y_val, val_pred[0])
    score2 = accuracy_score(y_val, val_pred[0])

    score3 = recall_score(y_val, val_pred[0])
    score4 = f1_score(y_val, val_pred[0])

    print('AUC_score :', score,'| Acc_score :', score2,'| recall_score :', score3,'| f1_score :', score4)
    gc.collect()

In [10]:
score = roc_auc_score(y_val, val_pred[0])
score2 = accuracy_score(y_val, val_pred[0])

score3 = recall_score(y_val, val_pred[0])
score4 = f1_score(y_val, val_pred[0])

print('precision_score :', score,'| Acc_score :', score2,'| recall_score :', score3,'| f1_score :', score4)
gc.collect()

NameError: name 'y_val' is not defined

## EDA

In [ ]:
# train.head()

In [ ]:
data_ = data.copy()
no_fe = lgbmgo(data_, train, test,rm_cols)

In [ ]:
train

## FE

In [9]:
data['dt_of_fr'] = pd.to_datetime(data['dt_of_fr'])
data['DT_hour'] = (data['dt_of_fr'].dt.hour).astype(np.int8)
#data['DT_Y'] = (data['dt_of_fr'].dt.year).astype(np.int8)
# data['DT_day_week'] = (data['dt_of_fr'].dt.dayofweek).astype(np.int8)
# data['DT_day_month'] = (data['dt_of_fr'].dt.day).astype(np.int8)
data['DT_M'] = (data['dt_of_fr'].dt.month).astype(np.int8)
# data = data.drop('dt_of_fr',axis = 1)
rm_cols += ['dt_of_fr']

In [10]:
data['dt_of_athrztn'].fillna(9999, inplace=True)
data['dt_of_athrztn_year'] = data['dt_of_athrztn'].astype(str).str[:4].astype(int)
#data['diff_athrztn_frY'] = data['DT_Y']-data['dt_of_athrztn_year']
# data = data.drop(['dt_of_athrztn','DT_Y'], axis = 1)
rm_cols += ['dt_of_athrztn']

In [11]:
bins = np.linspace(1600,2019, 20)
data['dt_of_athrztn_year2'] = np.digitize(data['dt_of_athrztn_year'], bins)

In [12]:
rm_cols

['id', 'dt_of_fr', 'dt_of_athrztn']

In [41]:
data_ = data.copy()
fe_time = lgbmgo(data_, train, test,rm_cols)


(61699, 184) (4398, 184) (61699,) (4398,) (2957, 184)
Training until validation scores don't improve for 300 rounds.
[100]	training's f1_score: 0.249038	valid_1's f1_score: 0.324222
[200]	training's f1_score: 0.449573	valid_1's f1_score: 0.457547
[300]	training's f1_score: 0.514635	valid_1's f1_score: 0.470053
[400]	training's f1_score: 0.541843	valid_1's f1_score: 0.467318
[500]	training's f1_score: 0.563178	valid_1's f1_score: 0.469708
[600]	training's f1_score: 0.580427	valid_1's f1_score: 0.472782
Early stopping, best iteration is:
[324]	training's f1_score: 0.522492	valid_1's f1_score: 0.475546
AUC_score : 0.6515463917525773 | Acc_score : 0.841518872214643 | recall_score : 0.3744075829383886 | f1_score : 0.4755455229495861


In [ ]:
data['danger1'] = data['no_tbc_zn_dstnc'] - data['tbc_rtl_str_dstnc']

In [ ]:
data['prevent1'] = data['cctv_in_100m'] + data['fr_wthr_fclt_in_100m']

In [ ]:
# rm_cols.remove('DT_Y')

In [ ]:
rm_cols

In [ ]:
# bins = np.linspace(-19175,9795, 300)
# data['danger1'] = np.digitize(data['danger1'], bins)

In [ ]:
# bins = np.linspace(0,16164, 160)
# data['no_tbc_zn_dstnc'] = np.digitize(data['no_tbc_zn_dstnc'], bins)

In [ ]:
# bins = np.linspace(0,24000, 240)
# data['tbc_rtl_str_dstnc'] = np.digitize(data['tbc_rtl_str_dstnc'], bins)

In [ ]:
data_ = data.copy()
fe_time_ath_ttl_dan_pre = lgbmgo(data_, train, test,rm_cols)

In [ ]:
# data['cctv_in_100m'][data['cctv_in_100m'] >= 1] = 1

In [ ]:
# data['fr_wthr_fclt_in_100m'][data['fr_wthr_fclt_in_100m'] >= 1] = 1

In [ ]:
# data_ = data.copy()
# fe_time_ath_ttl_dan_pre = lgbmgo(data_, train, test)

In [42]:
    train = data[:len(train)+2500]
    val = data[len(train)+2500:-len(test)]
    test = data[-len(test):]
    dataset = [train, val, test]

    for df in dataset:
        categorical_cols1 = df.select_dtypes(['object']).columns
        for col in categorical_cols1:
            df[col] = pd.Categorical(df[col]).codes
    train.drop(rm_cols,1,inplace=True)
    val.drop(rm_cols,1,inplace=True)
    test.drop(rm_cols,1,inplace=True)
    X_train = train.drop('fr_yn', 1)
    y_train = train['fr_yn']
    X_val = val.drop('fr_yn', 1)
    y_val = val['fr_yn']
    X_test = test.drop(columns = ['fr_yn'])
    
    print(X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape)
    #모델 생성 및 학습
    params = {
                    'boosting_type':'gbdt',
                    'n_jobs':-1,
                    'learning_rate':0.01,
                    'num_leaves': 250,
                    'max_depth':8,
                    'n_estimators':10000,
                    'seed' :SEED,
                }
    
    lgb_train = lgb.Dataset(X_train ,label= y_train)
    lgb_eval = lgb.Dataset(X_val, label= y_val)
    y_prob = np.zeros((X_test.shape[0]))

    model = lgb.train(params, lgb_train, feval=lgb_f1_score, valid_sets=[lgb_train, lgb_eval], early_stopping_rounds=100, verbose_eval=100)
    
    y_pred = model.predict(X_val)[:]
    val_pred = pd.DataFrame(y_pred)
    val_pred[0] = val_pred[0]>0.5
    val_pred[0] = val_pred[0].map({False:0,True:1})

    score = precision_score(y_val, val_pred[0])
    score2 = accuracy_score(y_val, val_pred[0])

    score3 = recall_score(y_val, val_pred[0])
    score4 = f1_score(y_val, val_pred[0])

    print('precision :', score,'| Acc_score :', score2,'| recall_score :', score3,'| f1_score :', score4)
    gc.collect()

(61699, 184) (1898, 184) (61699,) (1898,) (2957, 184)
Training until validation scores don't improve for 100 rounds.
[100]	training's f1_score: 0.249038	valid_1's f1_score: 0.324649
[200]	training's f1_score: 0.449573	valid_1's f1_score: 0.469218
[300]	training's f1_score: 0.514635	valid_1's f1_score: 0.477419
[400]	training's f1_score: 0.541843	valid_1's f1_score: 0.472089
Early stopping, best iteration is:
[324]	training's f1_score: 0.522492	valid_1's f1_score: 0.481541
precision : 0.6607929515418502 | Acc_score : 0.8298208640674394 | recall_score : 0.3787878787878788 | f1_score : 0.48154093097913325


261

In [13]:
def make_predictions(tr_df, tt_df, features_columns, target, lgb_params, NFOLDS=2):
    folds = KFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)

    X,y = tr_df[features_columns], tr_df[target]    
    P,P_y = tt_df[features_columns], tt_df[target]  

    tt_df = tt_df[[target]]    
    predictions = np.zeros(len(tt_df))
    oof = np.zeros(len(tr_df))
    
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
        print('Fold:',fold_)
        tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx]
        vl_x, vl_y = X.iloc[val_idx,:], y[val_idx]
            
        print(len(tr_x),len(vl_x))
        tr_data = lgb.Dataset(tr_x, label=tr_y)

        if LOCAL_TEST:
            vl_data = lgb.Dataset(P, label=P_y) 
        else:
            vl_data = lgb.Dataset(vl_x, label=vl_y)  

        estimator = lgb.train(
            lgb_params,
            tr_data,
            feval=lgb_f1_score,
            valid_sets = [tr_data, vl_data],
            verbose_eval = 200,
        )   
        
        pp_p = estimator.predict(P)
        predictions += pp_p/NFOLDS
        oof_preds = estimator.predict(vl_x)
        oof[val_idx] = (oof_preds - oof_preds.min())/(oof_preds.max() - oof_preds.min())
        

        if LOCAL_TEST:
            feature_imp = pd.DataFrame(sorted(zip(estimator.feature_importance(),X.columns)), columns=['Value','Feature'])
            print(feature_imp)
        
        del tr_x, tr_y, vl_x, vl_y, tr_data, vl_data
        gc.collect()
        
    tt_df['prediction'] = predictions
    print('OOF f1score:', f1_score(y, oof))
    
    return tt_df


In [43]:
y = model.predict(X_test)


In [44]:
val_pred = pd.DataFrame(y)
val_pred[0] = val_pred[0]>0.5
val_pred[0] = val_pred[0].map({False:0,True:1})

In [45]:
sub['fr_yn'] = val_pred[0]
sub['fr_yn'] = sub['fr_yn'].map({0:'N', 1:'Y'})
sub.to_csv('tmddnjs94_1128_nofe진우NEW 101.csv', index=False)